In [1]:
import logging
from numpy import random
import re    # for regular expressions 
import nltk  # for text manipulation 
import string 
import warnings 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt  
from wordcloud import WordCloud # data visualization library
#text features can be constructed using assorted techniques – Bag of Words, TF-IDF, and Word Embeddings.
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #tfidf and Bag-of-Words Features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from nltk.corpus import stopwords
import gensim # library for word2vec
%matplotlib inline
pd.set_option("display.max_colwidth", 200) 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
data = pd.read_excel('Description and merchant field to map category - RAK.xlsx')

In [4]:
# del data['txn_id']

In [8]:
data.head()

,txn_id,description,payee_merchant,Category
0,HPATM20171227101224090752085HP,NaN,ATM: UBI MID CORPORATE BR INDORE IN,Category not found
1,HPVCL2018125133122718952586HP,"PROCESS AS PER REQUEST, PLS SUBMIT UR TRAVEL CLAIM FOR EARLIER PERIOD TO PROCESS NEW MONEY REQUEST",Wallet load,Category not found
2,HPVCT201812610509887694821HP,NaN,Fuel: Daya Petroleum Calicut IN,Category not found
3,HPVCT201812610513668632314HP,NaN,Surcharge- Fuel: Daya Petroleum Calicut IN,Category not found
4,HPATM20183304213425556433HP,NaN,ATM: +CORP SEC-31 NOIDA NOIDA IN,Category not found


In [9]:
data['description'] = data['description'].str.replace("[^a-zA-Z]"," ")

In [10]:
data['payee_merchant'] = data['payee_merchant'].str.replace("[^a-zA-Z]"," ")

In [11]:
data = data[~data['payee_merchant'].str.contains('ATM', na = False)]

In [12]:
data["expense_description"] = data["description"].map(str)+ " " +data["payee_merchant"].map(str)

/home/krishna/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
data['expense_description'] = data['expense_description'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

/home/krishna/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
data['expense_description'] = data['expense_description'].str.replace('nan', '')

/home/krishna/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
STOPWORDS = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower() # lowercase text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [23]:
data['expense_description'] = data['expense_description'].apply(clean_text)

/home/krishna/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
data = data.reset_index(drop=True)

In [25]:
data.head()

,txn_id,description,payee_merchant,Category,expense_description
0,HPVCL2018125133122718952586HP,PROCESS AS PER REQUEST PLS SUBMIT UR TRAVEL CLAIM FOR EARLIER PERIOD TO PROCESS NEW MONEY REQUEST,Wallet load,Category not found,process per request pls submit travel claim earlier period process new money request wallet load
1,HPVCT201812610509887694821HP,NaN,Fuel Daya Petroleum Calicut IN,Category not found,fuel daya petroleum calicut
2,HPVCT201812610513668632314HP,NaN,Surcharge Fuel Daya Petroleum Calicut IN,Category not found,surcharge fuel daya petroleum calicut
3,HPCTU2018224161563211041797HP,airport to residence,Meru Cabs Head Office,Resident to station,airport residence meru cabs head office
4,HPVCT2018225641309134132757HP,NaN,Fuel SHANKAR AGENCIES TIRUNELVELI IN,Category not found,fuel shankar agencies tirunelveli


In [26]:
# tokenized_description = data['expense_description'].apply(lambda x: x.split()) # tokenizing
# tokenized_description.head()

In [27]:
# %%time
# from nltk.stem.porter import *
# stemmer = PorterStemmer()
# tokenized_description = tokenized_description.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [28]:
# for i in range (len(tokenized_description)):
#     tokenized_description[i] = ' '.join(tokenized_description[i])
# data['expense_description'] = tokenized_description
# data.head()

In [29]:
# data.isnull().sum()
# data

In [30]:
# data.to_csv('Processed_data.csv', encoding='utf-8', index=False)

In [31]:
data.Category.value_counts()

Category not found                        82805
Daily Field Allowance                     28718
Boarding                                   5983
Local Conveyance                           5488
Resident to station                        3172
Others                                     2952
Tickets (Bus , Train, Local transport)     1802
Toll/Parking Charges                       1606
Lodging                                    1470
Fuel Charges                                987
Travel by own vehicle                       672
Courier                                     670
Telephone/Telefax/Internet                  477
Residence from station to return            432
Driver Allowance                            369
Meal Allowances                             349
Fare (All Tickets for bus,train etc.)       346
Printing Charges                            184
Freight Outward                             137
Xerox                                       126
Sales Promotion Expenses                

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139016 entries, 0 to 139015
Data columns (total 5 columns):
txn_id                 139016 non-null object
description            119069 non-null object
payee_merchant         138993 non-null object
Category               139016 non-null object
expense_description    139016 non-null object
dtypes: object(5)
memory usage: 5.3+ MB


In [33]:
data.columns = data.columns.str.strip()

In [34]:
data.head()

,txn_id,description,payee_merchant,Category,expense_description
0,HPVCL2018125133122718952586HP,PROCESS AS PER REQUEST PLS SUBMIT UR TRAVEL CLAIM FOR EARLIER PERIOD TO PROCESS NEW MONEY REQUEST,Wallet load,Category not found,process per request pls submit travel claim earlier period process new money request wallet load
1,HPVCT201812610509887694821HP,NaN,Fuel Daya Petroleum Calicut IN,Category not found,fuel daya petroleum calicut
2,HPVCT201812610513668632314HP,NaN,Surcharge Fuel Daya Petroleum Calicut IN,Category not found,surcharge fuel daya petroleum calicut
3,HPCTU2018224161563211041797HP,airport to residence,Meru Cabs Head Office,Resident to station,airport residence meru cabs head office
4,HPVCT2018225641309134132757HP,NaN,Fuel SHANKAR AGENCIES TIRUNELVELI IN,Category not found,fuel shankar agencies tirunelveli


In [35]:
test = data[data['Category'].isin(['Category not found','Others'])]

In [36]:
test.to_csv('test_data.csv', encoding='utf-8', index=False)

In [37]:
train = data[~data['Category'].isin(['Category not found','Others'])]

In [38]:
train.to_csv('train_data.csv', encoding='utf-8', index=False)

In [39]:
train = pd.read_csv('train_data.csv')

In [40]:
test.head()

,txn_id,description,payee_merchant,Category,expense_description
0,HPVCL2018125133122718952586HP,PROCESS AS PER REQUEST PLS SUBMIT UR TRAVEL CLAIM FOR EARLIER PERIOD TO PROCESS NEW MONEY REQUEST,Wallet load,Category not found,process per request pls submit travel claim earlier period process new money request wallet load
1,HPVCT201812610509887694821HP,NaN,Fuel Daya Petroleum Calicut IN,Category not found,fuel daya petroleum calicut
2,HPVCT201812610513668632314HP,NaN,Surcharge Fuel Daya Petroleum Calicut IN,Category not found,surcharge fuel daya petroleum calicut
4,HPVCT2018225641309134132757HP,NaN,Fuel SHANKAR AGENCIES TIRUNELVELI IN,Category not found,fuel shankar agencies tirunelveli
23,HPVCT2018224102717887296487HP,NaN,HOTEL JAYA KOZHIKODE IN,Category not found,hotel jaya kozhikode


In [41]:
train1 = train[['expense_description','Category']]
# train1[train1['Category'].isin(['Sales Promotion Expenses'])]

In [42]:
train1.head()

,expense_description,Category
0,airport residence meru cabs head office,Resident to station
1,super tile emporium camelia sales tirupati impex local delhi,Daily Field Allowance
2,avni tiles tile art office sunny sanitation local delhi,Daily Field Allowance
3,avni tiles aggrawal kamakshi mkt vijay laxmi local delhi,Daily Field Allowance
4,avni tiles sanitary aggrawal marble bhatia associates vijay bath local delhi,Daily Field Allowance


In [37]:
train['Category'] = train['Category'].replace({'Local Conveyance': 'Travelling Expenses', 'Resident to station': 'Travelling Expenses', 'Tickets (Bus , Train, Local transport)': 'Travelling Expenses', 'Toll/Parking Charges': 'Travelling Expenses', 'Fuel Charges': 'Travelling Expenses', 'Travel by own vehicle': 'Travelling Expenses', 'Residence from station to return': 'Travelling Expenses', 'Driver Allowance': 'Travelling Expenses', 'Fare (All Tickets for bus,train etc.)': 'Travelling Expenses', 'Freight Outward': 'Travelling Expenses', 'Travel by Own Vehicle': 'Travelling Expenses', 'Travelling Expenses': 'Travelling Expenses', 'Vehicle Hire Charges': 'Travelling Expenses', 'Transit Allowance': 'Travelling Expenses'})

In [38]:
train['Category'] = train['Category'].replace({'Telephone/Telefax/Internet': 'Staff Welfare Expenses', 'Printing Charges': 'Staff Welfare Expenses', 'Xerox': 'Staff Welfare Expenses', 'Staff Welfare Expns': 'Staff Welfare Expenses', 'Internet Charges': 'Staff Welfare Expenses', 'Miscellaneous Expenses': 'Staff Welfare Expenses', 'Entertainment Expenses': 'Staff Welfare Expenses'})

In [39]:
train.Category.value_counts()

Daily Field Allowance       28718
Travelling Expenses         15142
Boarding                     5983
Lodging                      1470
Staff Welfare Expenses        844
Courier                       670
Meal Allowances               349
Sales Promotion Expenses       83
Name: Category, dtype: int64

In [40]:
train.to_csv('train_data_processed.csv', encoding='utf-8', index=False)